<a href="https://colab.research.google.com/github/tomdkt/test-gpu-llama/blob/main/base_semantic_segment_completion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

gemma-3-27b-it-Q3_K_L.gguf:

In [ ]:
!cat /etc/os-release
!nvcc --version
import torch
print(torch.cuda.is_available())

PRETTY_NAME="Ubuntu 22.04.4 LTS"
NAME="Ubuntu"
VERSION_ID="22.04"
VERSION="22.04.4 LTS (Jammy Jellyfish)"
VERSION_CODENAME=jammy
ID=ubuntu
ID_LIKE=debian
HOME_URL="https://www.ubuntu.com/"
SUPPORT_URL="https://help.ubuntu.com/"
BUG_REPORT_URL="https://bugs.launchpad.net/ubuntu/"
PRIVACY_POLICY_URL="https://www.ubuntu.com/legal/terms-and-policies/privacy-policy"
UBUNTU_CODENAME=jammy
nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2024 NVIDIA Corporation
Built on Thu_Jun__6_02:18:23_PDT_2024
Cuda compilation tools, release 12.5, V12.5.82
Build cuda_12.5.r12.5/compiler.34385749_0
True


In [ ]:
# Compile CUDA Llama-cpp-python (~ 25 minutes)
# !pip cache purge  # or pip cache remove llama-cpp-python
# !pip uninstall llama-cpp-python
# !CMAKE_ARGS="-DGGML_CUDA=on" FORCE_CMAKE=1 pip install llama-cpp-python


# import os
# import llama_cpp

# lib_path = os.path.dirname(llama_cpp.__file__)
# zip_path = "/content/llama_cpp.zip"
# print(f"Zipping from: {lib_path}")
# !zip -r /content/llama_cpp.zip "$lib_path"

# # 3. (Opcional) download zip(~26mb)
# from google.colab import files
# files.download(zip_path)

# from llama_cpp import llama_cpp
# # gpu enable must be True
# print(llama_cpp.llama_supports_gpu_offload())

In [ ]:
# (Optional) Upload the built Llama.cpp distribution(~25mb)
from google.colab import files
uploaded = files.upload()  # select `llama_cpp_a100.zip`
!unzip -o llama_cpp_L4.zip '*/llama_cpp/*' -d temp_llama
!cp -r temp_llama/usr/local/lib/python3.11/dist-packages/llama_cpp /usr/local/lib/python3.11/dist-packages/


!ls /usr/local/lib/python3.11/dist-packages/llama_cpp
!pip install diskcache
import sys
sys.path.append("/usr/local/lib/python3.11/dist-packages")

from llama_cpp import llama_cpp
# gpu enable must be True
print(llama_cpp.llama_supports_gpu_offload())


ggml_cuda_init: GGML_CUDA_FORCE_MMQ:    no
ggml_cuda_init: GGML_CUDA_FORCE_CUBLAS: no
ggml_cuda_init: found 1 CUDA devices:
  Device 0: NVIDIA L4, compute capability 8.9, VMM: yes


True


In [ ]:
# clean GPU RAM
# from numba import cuda

# device = cuda.get_current_device()
# device.reset()

# from accelerate import Accelerator
# accelerator = Accelerator()
# accelerator.free_memory()

# wait

In [ ]:
from llama_cpp import Llama

import os
os.environ["LLAMA_CPP_LOG_LEVEL"] = "ERROR"

llm = Llama.from_pretrained(
	repo_id="lmstudio-community/gemma-3-27b-it-GGUF",
	filename="gemma-3-27b-it-Q3_K_L.gguf",
	n_ctx=8192,
  n_threads=3,
  n_gpu_layers=70,
)

In [ ]:
def handle_response(response):
  # Display initial HTML structure for dynamic updates
  display(HTML("<b>Resposta:</b> <div id='response'></div><br><br><br>"))

  for chunk in response:
      content = chunk["choices"][0]["delta"].get("content", "")

      # Update the HTML content with the latest response chunk
      # Using `.format()` instead of f-strings to avoid backslash issues
      display(HTML("<script>document.getElementById('response').innerHTML += '<span style=\"font-size:20px\">{}</span>'; </script>".format(content.replace('`', '\\`'))), display_id='response_update')

In [ ]:
#test it
from IPython.display import display, HTML

response = llm.create_chat_completion(
    messages=[
        {
            "role": "user",
            "content": [
                {
                    "type": "text",
                    "text": "oi!!! como vc está?"
                }
            ]
        }
    ],
    stream=True,
    temperature=0.1,
    top_k=40,
    top_p=0.95,
    min_p=0.05,
    repeat_penalty=1.1,
    max_tokens=512
)

handle_response(response)

llama_perf_context_print:        load time =     396.46 ms
llama_perf_context_print: prompt eval time =     396.11 ms /    15 tokens (   26.41 ms per token,    37.87 tokens per second)
llama_perf_context_print:        eval time =    3780.09 ms /    54 runs   (   70.00 ms per token,    14.29 tokens per second)
llama_perf_context_print:       total time =    4568.42 ms /    69 tokens


In [ ]:
!pip install faiss-cpu langchain langchain-community sentence-transformers

In [ ]:
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.docstore.document import Document

from google.colab import files

uploaded = files.upload()
filename = list(uploaded.keys())[0]


Saving test.txt to test.txt


In [ ]:
from langchain.text_splitter import MarkdownHeaderTextSplitter

with open(filename, "r", encoding="utf-8") as f:
    texto = f.read()

splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
chunks = splitter.split_text(texto)


splitter = MarkdownHeaderTextSplitter(headers_to_split_on=[("##", "title")])
chunks = splitter.split_text(texto)

embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")
db = FAISS.from_documents(chunks, embeddings)

In [ ]:
# Split text into chapters

title = "A quarta edição"
chapter = [doc for doc in db.docstore._dict.values() if doc.metadata.get("title") == title]
context = "\n".join(doc.page_content for doc in chapter)
response = llm.create_chat_completion(
    messages=[
        {"role": "system", "content": "Você resume livros e deve responder perguntas sobre este capitulo de titulo " + title + ". Este é o conteudo do capitulo: " + context },
        {"role": "user", "content": "Qual o capitulo atual???!"}
    ],
    stream=True,
    temperature=0.1,
    #max_tokens=30,
)

handle_response(response)

Llama.generate: 25 prefix-match hit, remaining 1070 prompt tokens to eval


llama_perf_context_print:        load time =     396.46 ms
llama_perf_context_print: prompt eval time =    1736.88 ms /  1070 tokens (    1.62 ms per token,   616.05 tokens per second)
llama_perf_context_print:        eval time =   16443.01 ms /   213 runs   (   77.20 ms per token,    12.95 tokens per second)
llama_perf_context_print:       total time =   18958.08 ms /  1283 tokens


In [ ]:
# run semantic search only within this chapter
subject = "Emplasto bras";
resultados = db.similarity_search(subject, k=5)
context = "\n".join([doc.page_content for doc in resultados])

response = llm.create_chat_completion(
    messages=[
        {"role": "system", "content": "Você é um resumidor de livros e deve responder perguntas sobre este capitulo de titulo " + title + ". Este é o conteudo do capitulo: " + context },
        {"role": "user", "content": "Qual o capitulo atual???!"}
    ],
    stream=True,
    temperature=0.1,
    #max_tokens=30,
)

handle_response(response)

Llama.generate: 19 prefix-match hit, remaining 6542 prompt tokens to eval


llama_perf_context_print:        load time =     396.46 ms
llama_perf_context_print: prompt eval time =   13069.06 ms /  6542 tokens (    2.00 ms per token,   500.57 tokens per second)
llama_perf_context_print:        eval time =   29009.87 ms /   317 runs   (   91.51 ms per token,    10.93 tokens per second)
llama_perf_context_print:       total time =   43252.38 ms /  6859 tokens
